# Stats for Figure 2

In [96]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..') # needed for relative file path import with jupyter
from src.cc import ccptpt
from numpy.random import randint
import h5py
import matplotlib.pyplot as plt

### Load data

In [97]:
spikes = h5py.File('../data/spikes.h5','r')
spk_t = np.array(spikes['spikes']['biophysical']['timestamps'])

nmda_evts = pd.read_csv('../data/DendEventTimes/nmda_spk_times.csv')
na_evts = pd.read_csv('../data/DendEventTimes/na_spk_times.csv')

### Group dendritic spike events by electrotonic distance percentile

In [98]:
# create dataframe of node ID and electrotonic percentile

# first combine all dendritic spike dataframes no no node ID is left out
node_elec = pd.concat([nmda_evts[['segmentID', 'Elec_distance']], 
                       na_evts[['segmentID', 'Elec_distance']]],axis=0)
node_elec = node_elec.drop_duplicates('segmentID')

# calculate percentile rank and bin into 10th percentile steps
node_elec['Elec_dist_prc'] = round(node_elec.Elec_distance.rank(pct=True),1)

# group dendritic events by electrotonic percentile
nmda_evts = nmda_evts.merge(node_elec[['segmentID', 'Elec_dist_prc']],how='inner',on='segmentID')
na_evts = na_evts.merge(node_elec[['segmentID', 'Elec_dist_prc']],how='inner',on='segmentID')

nmda_evts = nmda_evts.groupby('Elec_dist_prc')['nmda_lower_bound_ms'].apply(list)
na_evts = na_evts.groupby('Elec_dist_prc')['na_lower_bound_ms'].apply(list)


TypeError: 'numpy.ndarray' object is not callable